In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [166]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import CNN
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import wandb
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt
import time
from qml_hep_lhc.layers import TwoLayerPQC
from qml_hep_lhc.layers.utils import get_count_of_qubits, get_num_in_symbols
from qml_hep_lhc.utils import _import_class
from qml_hep_lhc.models.metrics import custom_accuracy, qAUC

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


In [167]:
args = argparse.Namespace()

# Data
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 64
args.validation_split = 0.05
args.labels_to_categorical = 1
args.num_conv_layers = 2
args.conv_dims = [4,4]
args.num_fc_layers = 2
args.fc_dims = [8, 8]

# Base Model
args.learning_rate = 1e-3
args.epochs = 5

In [168]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤════════════════════╤══════════════════╤═══════════════════╤═════════════╕
│ Data   │ Train size         │ Val size         │ Test size         │ Dims        │
╞════════╪════════════════════╪══════════════════╪═══════════════════╪═════════════╡
│ X      │ (12031, 28, 28, 1) │ (634, 28, 28, 1) │ (2115, 28, 28, 1) │ (28, 28, 1) │
├────────┼────────────────────┼──────────────────┼───────────────────┼─────────────┤
│ y      │ (12031, 2)         │ (634, 2)         │ (2115, 2)         │ (2,)        │
╘════════╧════════════════════╧══════════════════╧═══════════════════╧═════════════╛

╒══════════════╤═══════╤════════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │    Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪════════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -1.47

In [169]:
data.config()

{'input_dims': (28, 28, 1), 'output_dims': (2,), 'mapping': [0, 1]}

In [170]:
cnn = CNN(data.config(), args)

In [171]:
cnn.compile()
cnn.build_graph().summary()

Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 4)         40        
                                                                 
 conv2d_10 (Conv2D)          (None, 28, 28, 4)         148       
                                                                 
 flatten_5 (Flatten)         (None, 3136)              0         
                                                                 
 dense_14 (Dense)            (None, 8)                 25096     
                                                                 
 dense_15 (Dense)            (None, 8)                 72        
                                                                 
 dense_16 (Dense)            (None, 2)                 18      

In [172]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]

In [173]:
cnn.fit(data, callbacks)

Epoch 1/5
188/188 [==============================] - 3s 15ms/step - loss: 0.0448 - auc_5: 0.9987 - accuracy: 0.9861 - val_loss: 0.0017 - val_auc_5: 1.0000 - val_accuracy: 0.9984 - lr: 0.0010
Epoch 2/5
188/188 [==============================] - 3s 14ms/step - loss: 0.0032 - auc_5: 0.9999 - accuracy: 0.9992 - val_loss: 3.5301e-04 - val_auc_5: 1.0000 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/5
188/188 [==============================] - 3s 15ms/step - loss: 0.0023 - auc_5: 1.0000 - accuracy: 0.9992 - val_loss: 3.5332e-04 - val_auc_5: 1.0000 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/5
188/188 [==============================] - 3s 14ms/step - loss: 0.0012 - auc_5: 1.0000 - accuracy: 0.9996 - val_loss: 0.0014 - val_auc_5: 1.0000 - val_accuracy: 0.9984 - lr: 0.0010
Epoch 5/5
188/188 [==============================] - 3s 17ms/step - loss: 4.8724e-04 - auc_5: 1.0000 - accuracy: 0.9999 - val_loss: 3.3668e-04 - val_auc_5: 1.0000 - val_accuracy: 1.0000 - lr: 0.0010


In [174]:
cnn.test(data, callbacks)

34/34 [==============================] - 0s 11ms/step - loss: 0.0295 - auc_5: 0.9991 - accuracy: 0.9991


[0.02951101027429104,
 0.9990521669387817,
 0.9990543723106384,
 0.9990521669387817]

In [175]:
args = argparse.Namespace()

# Data
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 64
args.validation_split = 0.05
args.hinge_labels = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 5

# Quantum layers
args.feature_map = 'AngleMap'
args.ansatz = 'TTN'
args.n_layers = 1

In [176]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Standardizing data...
Hinge labels...
Hinge labels...

Dataset :MNIST
╒════════╤════════════════════╤══════════════════╤═══════════════════╤═════════════╕
│ Data   │ Train size         │ Val size         │ Test size         │ Dims        │
╞════════╪════════════════════╪══════════════════╪═══════════════════╪═════════════╡
│ X      │ (12031, 28, 28, 1) │ (634, 28, 28, 1) │ (2115, 28, 28, 1) │ (28, 28, 1) │
├────────┼────────────────────┼──────────────────┼───────────────────┼─────────────┤
│ y      │ (12031,)           │ (634,)           │ (2115,)           │ (1,)        │
╘════════╧════════════════════╧══════════════════╧═══════════════════╧═════════════╛

╒══════════════╤═══════╤════════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │    Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪════════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -1.47 │ 112.53 │  -0    │  0.89 │ [0, 5626]  

In [177]:
data.config()

{'input_dims': (28, 28, 1), 'output_dims': (1,), 'mapping': [0, 1]}

In [178]:
input_dims = data.config()['input_dims']

In [179]:
model = models.Sequential()

for l in cnn.convs:
    model.add(l)
    
model.add(cnn.flatten)

for l in cnn.fcs[:-1]:
    model.add(l)

In [180]:
for l in model.layers:
    l.trainable = False

In [181]:
in_shape = [None] + list(input_dims)
print(in_shape)

cnn_out_shape = model.compute_output_shape(in_shape)[1:][0]
print(cnn_out_shape)

n_qubits = get_count_of_qubits(args.feature_map,cnn_out_shape)
n_inputs = get_num_in_symbols(args.feature_map,cnn_out_shape)

print(n_qubits, n_inputs)

[None, 28, 28, 1]
8
8 8


In [182]:
feature_map = _import_class(f"qml_hep_lhc.encodings.{args.feature_map}")()
ansatz = _import_class(f"qml_hep_lhc.ansatzes.{args.ansatz}")()

vqc = TwoLayerPQC(
    n_qubits,
    n_inputs,
    feature_map,
    ansatz,
    args.n_layers,
)

In [183]:
model.add(vqc)

In [184]:
model.compile(optimizer=optimizers.Adam(learning_rate=args.learning_rate),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[qAUC(), custom_accuracy])

In [185]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 28, 28, 4)         40        
                                                                 
 conv2d_10 (Conv2D)          (None, 28, 28, 4)         148       
                                                                 
 flatten_5 (Flatten)         (None, 3136)              0         
                                                                 
 dense_14 (Dense)            (None, 8)                 25096     
                                                                 
 dense_15 (Dense)            (None, 8)                 72        
                                                                 
 TwoLayerPQC (TwoLayerPQC)   (None, 1)                 30        
                                                                 
Total params: 25,386
Trainable params: 22
Non-trainab

In [186]:
model.fit(data.train_ds, callbacks = callbacks, epochs = args.epochs)

Epoch 1/5
188/188 [==============================] - 11s 57ms/step - loss: 0.6689 - q_auc_5: 0.9829 - custom_accuracy: 0.9865 - lr: 0.0010
Epoch 2/5
188/188 [==============================] - 11s 58ms/step - loss: 0.0723 - q_auc_5: 0.9975 - custom_accuracy: 0.9896 - lr: 0.0010
Epoch 3/5
188/188 [==============================] - 11s 57ms/step - loss: 0.0271 - q_auc_5: 0.9993 - custom_accuracy: 0.9933 - lr: 0.0010
Epoch 4/5
188/188 [==============================] - 11s 58ms/step - loss: 0.0149 - q_auc_5: 0.9995 - custom_accuracy: 0.9959 - lr: 0.0010
Epoch 5/5
188/188 [==============================] - 11s 58ms/step - loss: 0.0118 - q_auc_5: 0.9996 - custom_accuracy: 0.9969 - lr: 0.0010


In [187]:
model.evaluate(data.test_ds)

34/34 [==============================] - 2s 41ms/step - loss: 0.0154 - q_auc_5: 0.9981 - custom_accuracy: 0.9954


[0.015427658334374428, 0.9980819225311279, 0.9954044222831726]